In [ ]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from pandasql import sqldf # pip install pandasql

pysqldf = lambda q: sqldf(q, globals())

In [ ]:
session_key = 9888

qualificacao_key = 9884

In [ ]:
# Carrega os dados das voltas

url_session = f"https://api.openf1.org/v1/laps?session_key={session_key}"
resposta_session = requests.get(url_session)
resposta_session.status_code

if resposta_session.status_code == 200:
  dados_voltas = resposta_session.json()
else:
  print(f"Erro na requisição: {resposta_session.status_code}")

def converter_listas_para_string(dados):
  dados_processados = []

  for i in dados:
      novo_item = i.copy()

      for chave, valor in novo_item.items():
          if isinstance(valor, list):
              novo_item[chave] = str(valor)

      dados_processados.append(novo_item)

  return dados_processados

# Aplicar a conversão
dados_processados = converter_listas_para_string(dados_voltas)
df_voltas = pd.DataFrame(dados_processados)

df_voltas

In [ ]:
# Carrega os dados dos pilotos

url_piloto = f"https://api.openf1.org/v1/drivers?session_key={session_key}"
resposta_piloto = requests.get(url_piloto)
resposta_piloto.status_code

if resposta_piloto.status_code == 200:
  dados_piloto = resposta_piloto.json()
  df_piloto = pd.DataFrame(dados_piloto)
  df_piloto_final = df_piloto[['driver_number', 'name_acronym', 'team_colour']].copy()
  # Adicionar # ao campo de cores
  df_piloto_final['team_colour'] = '#' + df_piloto_final['team_colour']
else:
  print(f"Erro na requisição: {resposta_piloto.status_code}")

df_piloto_final

In [ ]:
# Carrega os dados de posição de chegada no grid

url = f"https://api.openf1.org/v1/session_result?session_key={session_key}"
resposta_position = requests.get(url)

if resposta_position.status_code == 200:
    df_posicao = pd.DataFrame(resposta_position.json())
    posicoes_finais = df_posicao.groupby('driver_number')['position'].last().sort_values()
    df_posicoes_finais = pd.DataFrame(posicoes_finais)
else:
    print(f"Erro: {resposta_position.status_code}")

df_posicoes_finais

In [ ]:
# Carrega os dados de qualificação

url_qualificacao = f"https://api.openf1.org/v1/starting_grid?session_key={qualificacao_key}"
resposta_qualificacao = requests.get(url_qualificacao)
resposta_qualificacao.status_code

if resposta_qualificacao.status_code == 200:
  dados_qualificacao = resposta_qualificacao.json()
  df_qualificacao = pd.DataFrame(dados_qualificacao)
else:
  print(f"Erro na requisição: {resposta_qualificacao.status_code}")

df_qualificacao

In [ ]:
consulta_1 = """

with w_pf_voltas as (
  select
    *
  from
    df_voltas
), w_pf_piloto as (
  select
    *
  from
    df_piloto_final
), w_df_posicoes_finais as (
  select
    driver_number,
    position as position_final
  from
    df_posicoes_finais
)
select
  w1.driver_number,
  w1.lap_number,
  w1.lap_duration,
  w2.name_acronym,
  w2.team_colour,
  w3.position_final,
  avg(w1.lap_duration) over (partition by w1.driver_number) as tempo_medio_volta
from
  w_pf_voltas w1,
  w_pf_piloto w2,
  w_df_posicoes_finais w3
where
  w1.driver_number = w2.driver_number
  and w1.driver_number = w3.driver_number

"""

# Pass the dataframes explicitly to sqldf and set the environment to locals()
voltas_final = sqldf(consulta_1, locals())
df_voltas_final = pd.DataFrame(voltas_final)
df_voltas_final

In [ ]:
# Cria o gráfico de boxplot com os tempos de volta dos pilotos e a média



# Ordem de chegada dos pilotos
ordem_pilotos = df_voltas_final.sort_values('position_final')['name_acronym'].unique()

# Dicionário mapeando cada piloto para sua cor de equipe
cores_pilotos = dict(zip(df_voltas_final['name_acronym'], df_voltas_final['team_colour']))

plt.figure(figsize=(14, 8))
ax = sns.boxplot(data=df_voltas_final, x='name_acronym', y='lap_duration',
                 order=ordem_pilotos,
                 showfliers=False,
                 showmeans=True,
                 meanprops={"marker": "o", "markerfacecolor": "red", "markeredgecolor": "red", "markersize": "3"},
                 palette=cores_pilotos,
                 legend=False,
                 hue='name_acronym')

plt.title(f'Ritmo de corrida - Sessão: {session_key}')
plt.xlabel('Piloto')
plt.ylabel('Tempo de volta')
plt.grid(True, alpha=0.3)
ax.yaxis.set_major_locator(MultipleLocator(0.5))  # Define intervalos de 0.5

# Adciona as médias no eixo
medias = df_voltas_final.groupby('name_acronym')['tempo_medio_volta'].first().reindex(ordem_pilotos)

for i, (piloto, media) in enumerate(medias.items()):
    ax.text(i, media, f'{media:.3f}',
            ha='center',
            va='bottom',
            fontsize=9)

plt.show()

In [ ]:
# Usando SQL eu filtro os 3 pilotos que fizeram a volta mais rápida

consulta_2 = """

with w_menor_tempo_piloto as (
  select
    name_acronym,
    lap_number,
    min(lap_duration) as lap_duration
  from
    df_voltas_final
  group by
    name_acronym
  order by
    lap_duration asc
)
select
  name_acronym as piloto,
  lap_number as num_volta,
  lap_duration as tempo_volta_rapida
from
  w_menor_tempo_piloto
limit
  3

"""

volta_rapida = pysqldf(consulta_2)
volta_rapida

In [ ]:
# Usando sql eu crio uma tabela que me mostra as posições que os pilotos ganharam ou perderam

consulta_3 = """

with w_qualificacao as (
  select
    driver_number as piloto,
    position as posicao_largada
  from
    df_qualificacao
), w_posicao as (
  select
    driver_number as piloto,
    position as posicao_chegada,
    points as pontos
  from
    df_posicao
), w_sigla_piloto as (
  select
    *
  from
    df_piloto_final
)
select
  w3.name_acronym as piloto,
  w1.posicao_largada,
  w2.posicao_chegada,
  case
    when (w1.posicao_largada - w2.posicao_chegada) > 0 then '+' || (w1.posicao_largada - w2.posicao_chegada)
    when (w1.posicao_largada - w2.posicao_chegada) < 0 then '-' || abs(w1.posicao_largada - w2.posicao_chegada)
    else 0
  end as ganho_posicoes,
  cast(w2.pontos as int) as pontos
from
  w_qualificacao w1,
  w_posicao w2,
  w_sigla_piloto w3
where
  w1.piloto = w2.piloto
  and w2.piloto = w3.driver_number

"""

ganho_posicoes = pysqldf(consulta_3)
ganho_posicoes

In [ ]:
# Gráfico de ganho de posições

# Primeiro, vamos converter a coluna 'ganho_posicoes' para numérico
ganho_posicoes['ganho_posicoes_num'] = ganho_posicoes['ganho_posicoes'].astype(float)

# Ordenar os dados pela posição de chegada
df_sorted = ganho_posicoes.sort_values('posicao_chegada', ascending=True)

# Cria o gráfico
plt.figure(figsize=(10, 8))
plt.axhspan(9.5, 0, alpha=0.2, color='gold') # Criar área de fundo para os 10 primeiros
cores = ['red' if x < 0 else 'green' for x in df_sorted['ganho_posicoes_num']] # Define as cores das barras
bars = plt.barh(df_sorted['piloto'], df_sorted['ganho_posicoes_num'], color=cores) # Define as barras
plt.xlabel('Posições')
plt.title(f'Ganho/Perda de Posições - Sessão: {session_key}')
plt.grid(True, alpha=0.3, )

# Adicionar os valores nas barras
for bar, valor in zip(bars, df_sorted['ganho_posicoes_num']):
    if valor is not None:
      plt.text(bar.get_width(),
              bar.get_y() + bar.get_height()/2,
              f'{valor:+.0f}' if valor != 0 else f'{valor:.0f}',
              ha='left' if valor >= 0 else 'right',
              va='center')

plt.show()